# Elaboration du fichier json d'entrainement

In [2]:
import pandas as pd
import json
import numpy as np

In [3]:
# Chagement du  fichier Frame

with open('frames\\frames.json') as fichier_frames:
    data = json.load(fichier_frames)
print(data[1]['turns'][0])

{'text': 'Hello, I am looking to book a vacation from Gotham City to Mos Eisley for $2100.', 'labels': {'acts': [{'args': [{'val': 'book', 'key': 'intent'}], 'name': 'inform'}, {'args': [{'val': 'Mos Eisley', 'key': 'dst_city'}, {'val': 'Gotham City', 'key': 'or_city'}, {'val': '2100', 'key': 'budget'}], 'name': 'inform'}, {'args': [], 'name': 'greeting'}], 'acts_without_refs': [{'args': [{'val': 'book', 'key': 'intent'}], 'name': 'inform'}, {'args': [{'val': 'Mos Eisley', 'key': 'dst_city'}, {'val': 'Gotham City', 'key': 'or_city'}, {'val': '2100', 'key': 'budget'}], 'name': 'inform'}, {'args': [], 'name': 'greeting'}], 'active_frame': 1, 'frames': [{'info': {'intent': [{'val': 'book', 'negated': False}], 'or_city': [{'val': 'Gotham City', 'negated': False}], 'dst_city': [{'val': 'Mos Eisley', 'negated': False}], 'budget': [{'val': '2100.0', 'negated': False}]}, 'frame_id': 1, 'requests': [], 'frame_parent_id': None, 'binary_questions': [], 'compare_requests': []}]}, 'author': 'user',

## Transformation du fichier frames en dataset avec les données utiles à LUIS

In [6]:
class create_luis_df:
    def __init__(self, frames_file):
        self.data = None
        self.df_data = None
        self.frames_file = frames_file
    
    def init_df_data(self):
        self.df_data = pd.DataFrame(columns=['text',
                                                    'or_city',
                                                    'from_start_pos',
                                                    'from_end_pos',
                                                    'dst_city',
                                                    'to_start_pos',
                                                    'to_end_pos',
                                                    'str_date',
                                                    'depart_date_start_pos',
                                                    'depart_date_end_pos',
                                                    'end_date',
                                                    'return_date_start_pos',
                                                    'return_date_end_pos',
                                                    'budgert',
                                                    'max_price_start_pos',
                                                    'max_price_end_pos'
                                            ])

    def get_data(self):
        with open(self.frames_file) as fichier_frames:
            self.data = json.load(fichier_frames)
    
    def get_pos(self, first_turn, keyword):
        start_pos, expression = self.get_start_pos(first_turn, keyword)
        end_pos = self.get_end_pos(start_pos, expression)
        return(start_pos, end_pos, expression)

    def get_start_pos(self, first_turn, keyword) -> int:
        if (type(first_turn['labels']['acts'])==list and len(first_turn['labels']['acts'])>1):
            for act in first_turn['labels']['acts']:
                if('args' in act):
                    dicts=act['args']
                    for dict in dicts:                  
                        if (dict['key']==keyword):
                            if('val' in dict):
                                expression = dict['val']
                                start_pos = first_turn['text'].find(expression)
                                if(start_pos>0):                    
                                    return (start_pos, expression)
                                else:
                                    return(None, None)
                            else:
                                print(dict)
        return(None, None)

    def get_end_pos(self, start_pos, expression) -> int:
        if(not start_pos == None):
            return (start_pos + len(expression))
        else:
            return None

    def charge_df_data(self):       
        for one_data in self.data:
            first_turn = one_data['turns'][0]
            text = one_data['turns'][0]['text']
            from_start_pos, from_end_pos, or_city = self.get_pos(first_turn, 'or_city')
            to_start_pos, to_end_pos, dst_city = self.get_pos(first_turn, 'dst_city')
            depart_date_start_pos, depart_date_end_pos, str_date = self.get_pos(first_turn, 'str_date')
            return_date_start_pos, return_date_end_pos, end_date = self.get_pos(first_turn, 'end_date')
            max_price_start_pos, max_price_end_pos, budget = self.get_pos(first_turn, 'budget')

            df_new_row = pd.DataFrame(  data=np.array([[
                                                        text,
                                                        or_city,
                                                        from_start_pos,
                                                        from_end_pos,
                                                        dst_city,
                                                        to_start_pos,
                                                        to_end_pos,
                                                        str_date,
                                                        depart_date_start_pos,
                                                        depart_date_end_pos,
                                                        end_date,                                                        
                                                        return_date_start_pos,
                                                        return_date_end_pos,
                                                        budget,
                                                        max_price_start_pos,
                                                        max_price_end_pos
                                                    ]]),
                                        columns=[
                                                    'text',
                                                    'or_city',
                                                    'from_start_pos',
                                                    'from_end_pos',
                                                    'dst_city',
                                                    'to_start_pos',
                                                    'to_end_pos',
                                                    'str_date',
                                                    'depart_date_start_pos',
                                                    'depart_date_end_pos',
                                                    'end_date',
                                                    'return_date_start_pos',
                                                    'return_date_end_pos',
                                                    'budget',
                                                    'max_price_start_pos',
                                                    'max_price_end_pos'
                                                ])
            self.df_data = pd.concat([self.df_data,df_new_row], ignore_index=True)
    
    def pipeline(self):
        self.init_df_data()
        self.get_data()
        self.charge_df_data()


monEssai = create_luis_df("frames\\frames.json")
monEssai.pipeline()

{'key': 'dst_city'}
{'key': 'dst_city'}
{'key': 'dst_city'}


In [7]:
monEssai.df_data.describe()

,text,or_city,from_start_pos,from_end_pos,dst_city,to_start_pos,to_end_pos,str_date,depart_date_start_pos,depart_date_end_pos,end_date,return_date_start_pos,return_date_end_pos,budgert,max_price_start_pos,max_price_end_pos,budget
count,1369,653,653,653,752,752,752,305,305,305,173,173,173,0,184,184,184
unique,1329,228,174,175,228,180,177,177,140,145,128,114,119,0,128,130,106
top,hi,Beijing,32,39,Punta Cana,16,26,August 27th,48,59,16,62,72,NaN,7,43,3200
freq,10,13,18,18,17,33,26,12,10,8,5,6,5,NaN,6,4,7


In [8]:
monEssai.df_data.head()

,text,or_city,from_start_pos,from_end_pos,dst_city,to_start_pos,to_end_pos,str_date,depart_date_start_pos,depart_date_end_pos,end_date,return_date_start_pos,return_date_end_pos,budgert,max_price_start_pos,max_price_end_pos,budget
0,I'd like to book a trip to Atlantis from Capri...,Caprica,41,48,Atlantis,27,35,"Saturday, August 13, 2016",52,77,None,None,None,NaN,117,121,1700
1,"Hello, I am looking to book a vacation from Go...",Gotham City,44,55,Mos Eisley,59,69,None,None,None,None,None,None,NaN,75,79,2100
2,Hello there i am looking to go on a vacation w...,None,None,None,Gotham City,63,74,None,None,None,None,None,None,NaN,None,None,None
3,"Hi I'd like to go to Caprica from Busan, betwe...",Busan,34,39,Caprica,21,28,"Sunday August 21, 2016",49,71,"Wednesday August 31, 2016",76,101,NaN,None,None,None
4,"Hello, I am looking to book a trip for 2 adult...",Kochi,106,111,Denver,116,122,None,None,None,None,None,None,NaN,67,74,"$21,300"


## Transformation du dataset en json exploitable par LUIS

In [ ]:
class create_luis_json:
    def __init__(
                    self,
                    df_frames,
                    intents_list            =['Book flight','Cancel','None'],
                    entities_list           =["From", "To", "Date_from", "Date_to", "Budget_max"],
                    composites_list         =[],
                    patternAnyEntities_list =[],
                    regex_entities_list     =[],
                    prebuiltEntities_list   =[],
                    model_features_list     =[],
                    regex_features_list     =[],
                    patterns_list           =[],
                    utterances_list         =[],
                ):
        self.json                       = None
        self.df_data                    = df_frames
        self.intents_list               = intents_list
        self.composites_list            = composites_list
        self.patternAnyEntities_list    = patternAnyEntities_list
        self.regex_entities_list        = regex_entities_list
        self.prebuiltEntities_list      = prebuiltEntities_list
        self.regex_features_list        = regex_features_list
        self.patterns_list              = patterns_list
        self.utterances_list            = utterances_list

    def luis_structure_python(self)->dict:
        struct_pyth = {}
        struct_pyth["luis_schema_version"]      = "3.2.0"
        struct_pyth["versionId"]                = "0.1"
        struct_pyth["name"]                     = "projet_10_fly_booking"
        struct_pyth["desc"]                     = "A LUIS model for booking a fly."
        struct_pyth["culture"]                  = "en-us"
        struct_pyth["tokenizerVersion"]         = "1.0.0"
        struct_pyth["intents"]                  = self.get_intents()
        struct_pyth["entities"]                 = self.get_entities()
        struct_pyth["composites"]               = self.get_composites()
        struct_pyth["patternAnyEntities"]       = self.get_patternAnyEntities()
        struct_pyth["regex_entities"]           = self.get_regex_entities()
        struct_pyth["prebuiltEntities"]         = self.get_prebuiltEntities()
        struct_pyth["model_features"]           = self.get_model_features()
        struct_pyth["regex_features"]           = self.get_regex_features()
        struct_pyth["patterns"]                 = self.get_patterns()
        struct_pyth["utterances"]               = self.get_utterances()
        struct_pyth["settings"]                 = self.get_settings()  

    def get_intents(self)->list:
        result = []        
        for intent in self.intents_list:
            w_dict = {}
            w_dict['name'] = intent
            result.append(w_dict)
        return result

    def get_entities(self)->list:
        result = []
        for entite in self.entities_list:
            w_dict = {}
            w_dict['name'] = entite
            result.append(w_dict)
        return result

    def get_composites(self)->list:
        result = []
        return result

    def get_patternAnyEntities(self)->list:
        result = []
        return result

    def get_regex_entities(self)->list:
        result = []
        return result

    def get_prebuiltEntities(self)->list:
        result = []
        return result

    def get_model_features(self)->list:
        result = []
        return result

    def get_regex_features(self)->list:
        result = []
        return result

    def get_patterns(self)->list:
        result = []
        return result

    def get_utterances(self)->list:
        result = []
        for one_data in self.df_data:
            w_dict = {}
            w_dict['text'] = one_data.text
            w_dict['intent'] = "Book flight"
            w_dict['entities'] = self.get_utterances_entities(one_data)
            # il faut coder la fonction get_utterances_entities qui permet de récupérer
            # les entities pour une phrase
        return result

    def get_settings(self)->list:
        result = []
        return result

    def save_jsons(self):
        pass

    def json_pipeline(self):
        pass

## Entrainement de LUIS

In [10]:
struct_pyth = {}
struct_pyth["luis_schema_version"] = "3.2.0"
struct_pyth["versionId"] = "0.1"
struct_pyth["name"] = "projet_10_fly_booking"
struct_pyth["desc"] = "A LUIS model for booking a fly."
struct_pyth["culture"] = "en-us"
struct_pyth["tokenizerVersion"] = "1.0.0"

In [13]:
pyt_to_json = json.dumps(struct_pyth)

In [14]:
print(pyt_to_json)

{"luis_schema_version": "3.2.0", "versionId": "0.1", "name": "projet_10_fly_booking", "desc": "A LUIS model for booking a fly.", "culture": "en-us", "tokenizerVersion": "1.0.0"}
